In [ ]:
import cv2 as cv
import youtube_dl
import src.network as network
import src.utility as ut

In [ ]:
weights = "yolov3.weights"
cfg = "yolov3.cfg"
classes = "coco.names"

url = "https://www.youtube.com/watch?v=zObiglKZKZc"

# Make sure to check that the quality exists on youtube.
video_quality = "480p"

# Any confidence under this threshold will be ignored.
threshold=0.7

# Eliminates overlapping (based on this threshold) boxes of the same class.
nms_threshold=0.4

# Yolo will downscale the image to this size. Smaller values will be faster. Remember; only multiples of 32!
size=(416, 416)

# Color of the boxes
box_color=(0, 0, 0)

# Color of the labels
text_color=(255, 255, 255)

# Scale of the font
font_scale=0.5

# Font
font=cv.FONT_HERSHEY_SIMPLEX

# This will adjust the thickess of the boxes, text and stuff
thickness=2

In [ ]:
# If you want to download different ones, just replace the url and the name

ut.download("https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg", ut.cfg_path, cfg)
ut.download("https://pjreddie.com/media/files/yolov3.weights", ut.weigths_path, weights)
ut.download("https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names", ut.class_path, classes)

In [ ]:
net = network.load(weights, cfg)
labels = network.prepare_classes(classes)

output_layers = network.extract_output_layers(net)

ydl_opts = {}
ydl = youtube_dl.YoutubeDL(ydl_opts)
info_dict = ydl.extract_info(url, download=False)
formats = info_dict.get('formats',None)


for f in formats:

    if f.get('format_note',None) == video_quality:

        url = f.get('url',None)
        cap = cv.VideoCapture(url)

        if not cap.isOpened():
            print('video not opened')
            exit(-1)

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            blob = network.image_to_blob(frame, size=size)
            outputs = network.forward(net, blob, output_layers)

            frame,_,_,_ = network.postprocess(frame, outputs, labels, threshold=threshold, nms_threshold=nms_threshold, box_color=box_color, text_color=text_color, font_scale=font_scale, font=cv.FONT_HERSHEY_SIMPLEX, thickness=2)

            cv.imshow('frame', frame)
            if cv.waitKey(30)&0xFF == ord('q'):
                break

        cap.release()

cv.destroyAllWindows()